In [88]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import timedelta
from datetime import date

fake = Faker()

In [89]:
n = 5  

billing = pd.DataFrame({
    "bill_id": range(1, n+1),
    "customer_id": np.random.randint(1, 10, size=n),
    "contract_id": np.random.randint(1, 10, size=n),
    "amount": np.round(np.random.uniform(50.0, 1000.0, size=n), 2),
    "bill_status_id": np.random.randint(1, 3, size=n),
    "issue_date": [fake.date_time_between(start_date="-2y", end_date="now") for _ in range(n)],
})


billing["due_date"] = billing["issue_date"] + pd.to_timedelta(7, unit="d")


billing["completed_date"] = [
    date + timedelta(days=np.random.randint(0, 31)) if np.random.rand() > 0.3 else None
    for date in billing["issue_date"]]

billing["created_at"] = [fake.date_time_between(start_date="-2y", end_date="now") for _ in range(n)]
billing["last_updated"] = billing["created_at"] + pd.to_timedelta(np.random.randint(0, 365, size=n), unit='d')


billing_status = pd.DataFrame({
    "bill_status_id": [1,2],
    "status": ["paid", "unpaid"]
})


In [90]:
billing["completed_date"] = billing["completed_date"].dt.date
billing["issue_date"] = billing["issue_date"].dt.date
billing["due_date"] = billing["due_date"].dt.date


billing["completed_date"] = billing["completed_date"].fillna(date(year=1995,day=1,month=1))
billing

,bill_id,customer_id,contract_id,amount,bill_status_id,issue_date,due_date,completed_date,created_at,last_updated
0,1,5,7,241.37,1,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892
1,2,7,1,462.67,2,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290
2,3,9,5,958.68,1,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191
3,4,6,9,390.98,2,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906
4,5,9,4,963.13,2,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817


In [91]:
billing = billing.merge(billing_status, on="bill_status_id", how="left")
billing

,bill_id,customer_id,contract_id,amount,bill_status_id,issue_date,due_date,completed_date,created_at,last_updated,status
0,1,5,7,241.37,1,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892,paid
1,2,7,1,462.67,2,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290,unpaid
2,3,9,5,958.68,1,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191,paid
3,4,6,9,390.98,2,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906,unpaid
4,5,9,4,963.13,2,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817,unpaid


In [92]:
billing.drop("bill_status_id",axis=1,inplace=True)
billing

,bill_id,customer_id,contract_id,amount,issue_date,due_date,completed_date,created_at,last_updated,status
0,1,5,7,241.37,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892,paid
1,2,7,1,462.67,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290,unpaid
2,3,9,5,958.68,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191,paid
3,4,6,9,390.98,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906,unpaid
4,5,9,4,963.13,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817,unpaid


In [93]:
billing["amount_billed"] = billing["amount"]
billing["amount_paid"] = billing.apply(lambda row: row["amount"] if row["status"] == "paid" else 0, axis=1)

billing

,bill_id,customer_id,contract_id,amount,issue_date,due_date,completed_date,created_at,last_updated,status,amount_billed,amount_paid
0,1,5,7,241.37,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892,paid,241.37,241.37
1,2,7,1,462.67,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290,unpaid,462.67,0.00
2,3,9,5,958.68,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191,paid,958.68,958.68
3,4,6,9,390.98,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906,unpaid,390.98,0.00
4,5,9,4,963.13,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817,unpaid,963.13,0.00


In [94]:
billing["is_paid"] = billing.apply(lambda row: True if row["status"]== "paid" else False, axis=1)

billing

,bill_id,customer_id,contract_id,amount,issue_date,due_date,completed_date,created_at,last_updated,status,amount_billed,amount_paid,is_paid
0,1,5,7,241.37,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892,paid,241.37,241.37,True
1,2,7,1,462.67,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290,unpaid,462.67,0.00,False
2,3,9,5,958.68,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191,paid,958.68,958.68,True
3,4,6,9,390.98,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906,unpaid,390.98,0.00,False
4,5,9,4,963.13,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817,unpaid,963.13,0.00,False


In [95]:
billing["days_overdue"] = billing.apply(lambda row: 0 if row["due_date"] >= row["completed_date"] else (row["completed_date"] -row["due_date"] ).days, axis=1 )

billing

,bill_id,customer_id,contract_id,amount,issue_date,due_date,completed_date,created_at,last_updated,status,amount_billed,amount_paid,is_paid,days_overdue
0,1,5,7,241.37,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892,paid,241.37,241.37,True,0
1,2,7,1,462.67,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290,unpaid,462.67,0.00,False,0
2,3,9,5,958.68,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191,paid,958.68,958.68,True,0
3,4,6,9,390.98,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906,unpaid,390.98,0.00,False,0
4,5,9,4,963.13,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817,unpaid,963.13,0.00,False,0


In [96]:
billing.rename(columns={"status": "bill_status"}, inplace=True)
billing.drop("amount",axis=1,inplace=True)
billing

,bill_id,customer_id,contract_id,issue_date,due_date,completed_date,created_at,last_updated,bill_status,amount_billed,amount_paid,is_paid,days_overdue
0,1,5,7,2024-02-21,2024-02-28,2024-02-26,2024-11-09 06:27:39.574892,2025-11-01 06:27:39.574892,paid,241.37,241.37,True,0
1,2,7,1,2025-04-19,2025-04-26,2025-04-23,2024-04-24 01:07:21.962290,2025-03-06 01:07:21.962290,unpaid,462.67,0.00,False,0
2,3,9,5,2025-06-21,2025-06-28,1995-01-01,2024-03-10 18:56:20.107191,2025-01-16 18:56:20.107191,paid,958.68,958.68,True,0
3,4,6,9,2024-09-08,2024-09-15,1995-01-01,2023-12-14 05:38:58.579906,2024-03-11 05:38:58.579906,unpaid,390.98,0.00,False,0
4,5,9,4,2024-10-13,2024-10-20,2024-10-18,2024-05-05 04:43:25.494817,2024-07-24 04:43:25.494817,unpaid,963.13,0.00,False,0
